# Hackathon 9

Images sourced from [Chris Olah's blog post on LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

Topics:
- TensorFlow RNNs and Cells
- LSTM

In today's demo, we'll teach an RNN how to speak English.

This is all setup in a IPython notebook so you can run any code you want to experiment with. Feel free to edit any cell, or add some to run your own code.

In [1]:
# We'll start with our library imports...
from __future__ import print_function

import random
import os  # to work with file paths

import tensorflow as tf         # to specify and run computation graphs
from tensorflow import keras
import numpy as np              # for numerical operations taking place outside of the TF graph
import matplotlib.pyplot as plt # to draw plots

#### RNN/LSTM theory recap

Recurrent neural networks (RNNs) are computation graphs with loops (i.e., not directed acyclic graphs). Because the backpropagation algorithm only works with DAGs, we have to unroll the RNN through time. Tensorflow provides code that handles this automatically.

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" width="80%">


The most common RNN unit is the LSTM, depicted below:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" width="80%">

We can see that each unit takes 3 inputs and produces 3 outputs, two which are forwarded to the same unit at the next timestep and one true output, $h_t$ depicted coming out of the top of the cell.

The upper right output going to the next timestep is the cell state. It carries long-term information between cells, and is calculated as: 

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png" width="80%">

where the first term uses the forget gate $f_t$ to decide to scale the previous state (potentially making it smaller to "forget" it), and the second term is the product of the update gate $i_t$ and the state update $\tilde{C}_t$. Each of the forget and update gates are activated with sigmoid, so their range is (0,1).

The true output and the second, lower output on the diagram are calculated by the output gate:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" width="80%">

First, $o_t$ is calculated from the output of the previous timestep concatenated with the current input, but then it's mixed with the cell state to get the true output. Passing on this output to the next timestep as the hidden state gives the unit a kind of short term memory.

(Images sourced from [Colah's Blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

Today, we're going to teach a recurrent model how to speak English by starting from a sequence of words and asking the model to predict what the next word should be. And what better way to learn English than by learning to talk like an angry media reviewer on the internet? We'll be using the IMDB review corpus for this task.

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17473536/17464789 [==============================] - 0s 0us/step


The data is encoded as a list of integers. We'll get the mapping of words to integer values in order to be able to translate the words back and forth.

In [3]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text if i != 0])

1654784/1641221 [==============================] - 0s 0us/step


Then we'll pad the text sequences so that they're all of identical length and easier to work with

In [4]:
SEQ_LEN = 256

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=SEQ_LEN)
train_targets = keras.preprocessing.sequence.pad_sequences(train_data[:,1:],
                                                        value=word_index["<START>"],
                                                        padding='post',
                                                        maxlen=SEQ_LEN)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=SEQ_LEN)
test_targets = keras.preprocessing.sequence.pad_sequences(test_data[:,1:],
                                                        value=word_index["<START>"],
                                                        padding='post',
                                                        maxlen=SEQ_LEN)

print("Training data has shape {}".format(train_data.shape))
print("Training targets has shape {}".format(train_targets.shape))
print("Testing data has shape {}".format(train_data.shape))
print("Testing targets has shape {}".format(test_targets.shape))

Training data has shape (25000, 256)
Training targets has shape (25000, 256)
Testing data has shape (25000, 256)
Testing targets has shape (25000, 256)


In [5]:
# visualize some of the data
idx = random.randrange(train_data.shape[0])
print(train_data[idx])
print(decode_review(train_data[idx]))
print(train_targets[idx])
print(decode_review(train_targets[idx]))

[   1   13  391    4  640  179   73   36   43  713   52    4  123    9
  527   13  391   12    5  198  160  527  155   44   12    4   64  646
  109   50  126   16   23    4  123   16   15   31    2   11   15   31
  390   21   95   13   67    4   85  390  587   15  390    5    4  123
    9  527   45   24  163   24  163   13   92  181   84    8  135   64
 1393   84   79   12   88   48  507   38 1393  138   81   36 1919   84
   36   92   60  124    5  135   15  507   24 1393   42 2772  195    8
  391   12   45   40  660    4 1749    9  767   21  115  267 1005   21
 3847   14    9  427    4  249  123   13   28  126  110   11   61  113
    4  640   26  394   13  384   25   70  391   98  507   43  527   41
 7096    9   55  835   41 7643  640    5   85  640   23    2    2   26
   66  995    5  633 7034    7   66   78  116  146   24  252   51  134
 1393   84   67   11   14  123   21 4235  409   54   36   92   60  124
  233   44  101    7  178  218  618    6 1393  931    0    0    0    0
    0 

Each datum is a string of up to 256 successive words from the corpus, and the target is a similar window, but shifted forward by one word. This is setup to train the model to, given a few preceding words, predict what the next word in the sequence will be.

Initially, in the data each word in the sequence is represented as an integer (notice the shape). This discrete representation fails to capture any semantic relationships between words. I.e., the model wouldn't know that "crimson" and "scarlet" are more similar than "red" and "blue". The solution is to learn an word embedding as the first part of the model to transform each integer into a relatively small, dense vector (as compared to a one-hot). Then, similar words will train to have similar embeddings.

We'll use [tf.nn.embedding_lookup](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) to do this which we provide a (usually trainable) VOCAB_SIZE x EMBEDDING_SIZE matrix.

In [6]:
VOCAB_SIZE = max(word_index.values())
EMBEDDING_SIZE = int(np.sqrt(VOCAB_SIZE))
print("Vocab size is {} and is embedded into {} dimensions".format(VOCAB_SIZE, EMBEDDING_SIZE))

# setup input and embedding
input_ph = tf.placeholder(tf.int32, [None, 256])
target_ph = tf.placeholder(tf.int32, [None, 256])

with tf.variable_scope("embedding", reuse=tf.AUTO_REUSE):
    embedding_matrix = tf.get_variable('embedding_matrix', dtype=tf.float32, shape=[VOCAB_SIZE, EMBEDDING_SIZE],
                                       trainable=True)
word_embeddings = tf.nn.embedding_lookup(embedding_matrix, input_ph)
print("The output of the word embedding: " + str(word_embeddings))

W1118 13:25:14.432789 47064002985504 deprecation.py:506] From /util/opt/anaconda/deployed-conda-envs/packages/tensorflow/envs/tensorflow-1.14.0-py27/lib/python2.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Vocab size is 88587 and is embedded into 297 dimensions
The output of the word embedding: Tensor("embedding_lookup/Identity:0", shape=(?, 256, 297), dtype=float32)


Now we want to declare an architecture that looks like this (replacing GRU with LSTM and char embedding with word embedding).

<img src="https://tensorflow.org/tutorials/sequences/images/text_generation_training.png" width="80%">

TensorFlow separates the declaration of [RNNCells](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/RNNCell) from the [RNNs](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) that run them. In the code below, we declare an [LSTM cell](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell), and create tensors for the inputs to the first unit. We use zeros for the initial hidden state and current state, but it's also possible to declare trainable variables for these as well.

In [7]:
LSTM_SIZE = 200 # number of units in the LSTM layer, this number taken from a "small" language model
BATCH_SIZE = 32

lstm_cell = tf.contrib.rnn.LSTMCell(LSTM_SIZE)

# Initial state of the LSTM memory.
initial_state = lstm_cell.zero_state(BATCH_SIZE, tf.float32)
print("Initial state of the LSTM: " + str(initial_state))

W1118 13:25:16.511984 47064002985504 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1118 13:25:16.514000 47064002985504 deprecation.py:323] From <ipython-input-7-75721b72b976>:4: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Initial state of the LSTM: LSTMStateTuple(c=<tf.Tensor 'LSTMCellZeroState/zeros:0' shape=(32, 200) dtype=float32>, h=<tf.Tensor 'LSTMCellZeroState/zeros_1:0' shape=(32, 200) dtype=float32>)


Then, we'll pass the newly declared cell and the training sequence of word embeddings to [tf.nn.dynamic_rnn](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) as the inputs over time to the LSTM. `dynamic_rnn` runs an `RNNCell` using an internal `while` loop, and returns the sequence of outputs from the LSTM at each timestep and the final state of the LSTM.

In [8]:
# setup RNN
outputs, state = tf.nn.dynamic_rnn(lstm_cell, word_embeddings,
                                   initial_state=initial_state,
                                   dtype=tf.float32)
print("The outputs over all timesteps: "+ str(outputs))
print("The final state of the LSTM layer: " + str(state))
logits = tf.layers.dense(outputs, VOCAB_SIZE)

W1118 13:25:16.563955 47064002985504 deprecation.py:323] From <ipython-input-8-4814f55054d6>:4: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W1118 13:25:16.742316 47064002985504 deprecation.py:506] From /util/opt/anaconda/deployed-conda-envs/packages/tensorflow/envs/tensorflow-1.14.0-py27/lib/python2.7/site-packages/tensorflow/python/ops/rnn_cell_impl.py:961: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1118 13:25:19.220468 47064002985504 deprecation.py:323] From <ipython-input-8-4814f55054d6>:7: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense inst

The outputs over all timesteps: Tensor("rnn/transpose_1:0", shape=(32, 256, 200), dtype=float32)
The final state of the LSTM layer: LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(32, 200) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(32, 200) dtype=float32>)


And to calculate the loss between two sequences, we'll import a function from [tf.contrib.seq2seq](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq) called [sequence_loss](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/sequence_loss). It calculates the weighted cross-entropy loss between the first two arguments, and the third argument provides weights for averaging. We weight uniformly here, but weights could also be calculated based on where in the sequence the target is (e.g., penalize less earlier in the sequence, but more later) or based on the content of the target (e.g., low weight on guessing articles correctly and larger weight on getting nouns and verbs correct).

We'll optimize using TensorFlow's [RMSProp](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer) optimizer, which requires an explicit learning rate, but otherwise as usual. We switch from the Adam optmizer because we don't want the adaptive learning rate feature, which can interact badly with the recurrent gradients.

In [9]:
LEARNING_RATE = 1e-4

loss = tf.contrib.seq2seq.sequence_loss(
    logits,
    target_ph,
    tf.ones([BATCH_SIZE, SEQ_LEN], dtype=tf.float32), # we'll use uniform weight over timesteps
    average_across_timesteps=True,
    average_across_batch=True)

optimizer = tf.train.RMSPropOptimizer(LEARNING_RATE)
train_op = optimizer.minimize(loss)

W1118 13:25:20.846189 47064002985504 deprecation.py:506] From /util/opt/anaconda/deployed-conda-envs/packages/tensorflow/envs/tensorflow-1.14.0-py27/lib/python2.7/site-packages/tensorflow/python/training/rmsprop.py:119: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Finally, we'll create a Session, initialize the variables, and run the train op once. This model is relatively heavyweight, so we don't want to optimize it on the login node.

In [10]:
session = tf.Session()
session.run(tf.global_variables_initializer())

# we'll just run one step, omitting the usual epoch code
_ = session.run(train_op, feed_dict={input_ph: train_data[idx:idx + BATCH_SIZE], target_ph: train_targets[idx:idx + BATCH_SIZE]})

Now, we'll try generating from the model.

In [11]:
# this value is to limit load on the login node, must be <= SEQ_LEN
GEN_LEN = 10

# generated_sequence needs to be a valid input
generated_sequence = np.array([[word_index["<START>"]] + [word_index["<PAD>"]]*(SEQ_LEN - 1)] * BATCH_SIZE) 
for idx in range(1, GEN_LEN):
    logits_val = session.run(logits, feed_dict={input_ph: generated_sequence})
    generated_words = np.array([np.argmax(l_val[idx]) for l_val in logits_val])
    generated_sequence[:,idx] = generated_words
    print(decode_review(generated_sequence[0]))

<START> topness
<START> topness jez
<START> topness jez topness
<START> topness jez topness wouldnt
<START> topness jez topness wouldnt vipers
<START> topness jez topness wouldnt vipers virginities
<START> topness jez topness wouldnt vipers virginities karega
<START> topness jez topness wouldnt vipers virginities karega virginities
<START> topness jez topness wouldnt vipers virginities karega virginities coolest


Create a 2 layer [LSTMCell](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell/LSTMCell) with an [Attention Wrapper](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/AttentionCellWrapper). Your code should use `train_ph` inputs as above and your code should finish with the `loss` tensor. This should be pretty straightforward with the TensorFlow documentation.

This model is very large and trains for a long time, so please don't try to optimize it in this notebook.

In [12]:
# Your code here
LSTM_SIZE = 200 # number of units in the LSTM layer, this number taken from a "small" language model
BATCH_SIZE = 64

cells = []
for i in range(2):                   
    cell = tf.contrib.rnn.LSTMCell(LSTM_SIZE, state_is_tuple=True)

    cell = tf.contrib.rnn.AttentionCellWrapper(
        cell, attn_length=40, state_is_tuple=True)
    
    cells.append(cell)

cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
outputs, state = tf.nn.dynamic_rnn(cell, word_embeddings,
                                   dtype=tf.float32)

logits = tf.layers.dense(outputs, VOCAB_SIZE)
LEARNING_RATE = 1e-4

loss = tf.contrib.seq2seq.sequence_loss(
    logits,
    target_ph,
    tf.ones([BATCH_SIZE, SEQ_LEN], dtype=tf.float32), # we'll use uniform weight over timesteps
    average_across_timesteps=True,
    average_across_batch=True)

W1118 13:26:37.631315 47064002985504 deprecation.py:323] From <ipython-input-12-b8840f857ea4>:14: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W1118 13:26:40.930468 47064002985504 deprecation.py:506] From /util/opt/anaconda/deployed-conda-envs/packages/tensorflow/envs/tensorflow-1.14.0-py27/lib/python2.7/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn_cell.py:104: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
